In [9]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install torch transformers datasets scikit-learn accelerate


In [10]:
#Importing
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    pipeline
)
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import torch


In [13]:
#Load Data set
dataset = load_dataset("imdb", split={'train': 'train[:1%]', 'test': 'test[:1%]'})
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.rename_column("label", "labels")
dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])


In [15]:
#Define Metrics
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="binary")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#Training
training_args = TrainingArguments(
    output_dir="./results",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    logging_dir="./logs"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    compute_metrics=compute_metrics
)


In [19]:
trainer.train()
metrics = trainer.evaluate()
print(metrics)


C:\Users\paulj\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


C:\Users\paulj\anaconda3\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.006575298495590687, 'eval_accuracy': 1.0, 'eval_f1': 0.0, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_runtime': 13.5233, 'eval_samples_per_second': 18.487, 'eval_steps_per_second': 2.366, 'epoch': 1.0}


C:\Users\paulj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\paulj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\paulj\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [25]:
!pip install --upgrade torch transformers datasets scikit-learn ipywidgets


   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   -------------------------------------- - 2.1/2.2 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 9.5 MB/s  0:00:00

  Attempting uninstall: widgetsnbextension

    Found existing installation: widgetsnbextension 3.6.6

    Uninstalling widgetsnbextension-3.6.6:

      Successfully uninstalled widgetsnbextension-3.6.6

  Attempting uninstall: jupyterlab_widgets

    Found existing installation: jupyterlab-widgets 1.0.0

   ------------- -------------------------- 1/3 [jupyterlab_widgets]
    Uninstalling jupyterlab-widgets-1.0.0:
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
      Successfully uninstalled jupyterlab-widgets-1.0.0
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
  Attempting uninstall: ipywidgets
   ------------- -------------------------- 1/3 [jupyterlab_widgets]
    Found existing installation: ipywidgets 7.8.1
   ----------

In [26]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"


In [29]:
#Promt 
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2", device=-1)

def gen_text(prompt):
    out = generator(
        prompt,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.1,
        pad_token_id=generator.tokenizer.eos_token_id
    )
    return out[0]["generated_text"]

print("--- Zero-shot ---")
print(gen_text("Explain how AI models understand language."))

print("\n--- Few-shot ---")
print(gen_text("Q: What is AI?\nA: Artificial Intelligence helps machines learn.\nQ: How do AI models understand language?\nA:"))

print("\n--- Chain-of-Thought ---")
print(gen_text("Q: How do AI models understand language?\nLet's think step-by-step:"))


Device set to use cpu


--- Zero-shot ---
Explain how AI models understand language.
The biggest difference between the two is that they can't tell a lot about languages, and so in fact it's not possible to directly explain why people are using them at all (for example by saying "I'm going with my English dictionary") but rather because those words have more meanings than their grammar does. The main problem here is what we need to do if you want to use something like

--- Few-shot ---
Q: What is AI?
A: Artificial Intelligence helps machines learn.
Q: How do AI models understand language?
A: They can solve a problem by looking at the data they have to parse and analyzing it, while learning about how things are working in general. And if you look at what people say that makes them think we know something or not, then I don't see this as an explanation for why there's such limited computational power available today. It does take more than just human brains doing everything but making decisions based

--- Chain